In [283]:
import pandas as pd
import numpy as np
import math
import time
import os
from functools import reduce
import re
from geopy.geocoders import Nominatim
from datetime import datetime

geolocator = Nominatim(user_agent="cmpt459")

In [78]:
#1.2 Clean age column

def clean_data(df):

  df.age.astype(str)

  # get all non empty age rows
  non_age_empty_cases = df.dropna(subset=['age'])

  # clean up age ranges
  mask = (non_age_empty_cases['age'].str.contains('-'))

  # convert age range to the middle age value
  def range_to_age(row):
    age_range = row.age.replace(' ', '').split('-')
    if any((not age.isnumeric() or age == '') for age in age_range):
     for age in age_range:
       if age.isnumeric():
         return age
    else:
      mean_age = reduce(lambda a, b: int(a) + int(b), age_range) / len(age_range)
      return mean_age 

  non_age_empty_cases.loc[mask, 'age'] = non_age_empty_cases[mask].apply(lambda row: range_to_age(row), axis=1)

  # convert babies/month old children to 1 years old if months < 12 
  non_age_empty_cases['age'] = non_age_empty_cases.age.astype(str)
  month_mask = (non_age_empty_cases['age'].str.contains('month'))

  def months_to_years(row):
    months = int(re.sub('[^0-9]','', row.age))
    return 1 if months < 12 else months/12

  non_age_empty_cases.loc[month_mask, 'age'] = non_age_empty_cases[month_mask].apply(lambda row: months_to_years(row), axis=1)

  # remove other symbols
  non_age_empty_cases['age'] = non_age_empty_cases.age.str.replace('+', '', regex=False)

  non_age_empty_cases['age'] = non_age_empty_cases.age.astype(float)

  # get all missing age values and fill them with mean age of the rest of the rows
  mean_age = non_age_empty_cases['age'].mean()

  empty_age_cases = df[df.age.isna()]
  empty_age_cases['age'] = mean_age
  empty_age_cases['age'] = empty_age_cases.age.astype(float)

  df = pd.concat([non_age_empty_cases, empty_age_cases])

  # impute missing sex

  non_empty_sex =  df.dropna(subset=['sex'])
  empty_sex = df[df.sex.isna()]

  # shuffle rows
  empty_sex = empty_sex.sample(frac=1)

  # assign half of empty sex cells as male, other half as female
  half_row_count = int(len(empty_sex.index)/2)

  empty_sex.iloc[:half_row_count].sex = 'female'
  empty_sex.iloc[half_row_count:].sex = 'male'

  #merge dfs again
  df = pd.concat([non_empty_sex, empty_sex])

  # shuffle rows
  df = df.sample(frac=1)

  # impute missing country

  # drop where countries and lat long are missing
  df = df.dropna(subset=['country', 'latitude', 'longitude'], how='all')
  
  non_empty_country =  df.dropna(subset=['country'])
  empty_country = df[df.country.isna()]
  
  def get_country(row):
    location = geolocator.reverse(','.join([str(row.latitude), str(row.longitude)]), language='en')
    return location.address.split(",")[-1]
  
  empty_country['country'] = empty_country.apply(func=get_country, axis=1)

  #merge
  df = pd.concat([non_empty_country, empty_country])
  
  df = df.sort_index()
  return df


In [108]:

#1.4: 

def transform_counties(df):
  us_mask = (df['Country_Region'] == 'US')
  non_us_country = df[~us_mask]
 
  us_country = df[us_mask]

  us_country = us_country.groupby(['Province_State']).mean().reset_index()

  us_country['Country_Region'] = 'United States'
  us_country['Combined_Key'] = us_country['Province_State'].apply(lambda province : province + ", United States")
  return pd.concat([us_country, non_us_country])




In [ ]:
# impute missing incidence,

In [317]:
#1.5: merge location and cases_train

def merge_df_location(cases_df, location_df):
    cases = cases_df.copy()
    cases['combined_key'] = cases.apply(lambda x: add_combined_key(x, location_df), axis=1)
    
    cases = pd.merge(cases, location_df, left_on=['combined_key'], right_on=['Combined_Key'], how="left")
    cases.drop(['Province_State', 'Country_Region', 'Lat', 'Long_', 'Last_Update'], axis=1)
    return cases

# def impute_location(location_df):
    

#   province_location_df = location_df.dropna(subset=['Province_State'])
#   empty_province_location_df = location_df[location_df.Province_State.isna()]
#   print(len(province_location_df.index))
#   print(len(empty_province_location_df.index))
#   print(len(cases_df.index))
#   merged_dataset_province = pd.merge(cases_df, province_location_df, left_on=['province'], right_on=['Province_State'], how="left")
#   merged_dataset_empty_province = pd.merge(cases_df, empty_province_location_df, left_on=['country'], right_on=['Country_Region'], how="left")    
#   merged_dataset = pd.concat([merged_dataset_province, merged_dataset_empty_province])
#   merged_dateset = merged_dataset.sort_index()
#   merged_dataset.drop(['Province_State', 'Country_Region', 'Lat', 'Long_', 'Last_Update'], axis=1)
# #   merged_dataset = cases_df.set_index('country').join(location_df.set_index('Country_Region'))
#   return merged_dataset

def add_combined_key(row, location_df):
    # no province case
    if location_df['Combined_Key'].eq(row['country']).any():
        return row['country']

    combined = str(row['province']) + ', ' + row['country']
    combined = combined.lower()
    if location_df['Combined_Key'].str.lower().str.contains(combined).any():
        return location_df[location_df['Combined_Key'].str.lower().str.contains(combined)].iloc[0]['Combined_Key']
    elif location_df['Province_State'].str.lower().str.contains(row['country'].lower()).any():
        return location_df[location_df['Province_State'].str.lower().str.contains(row['country'].lower(), na=False)].iloc[0]['Combined_Key']
    elif location_df['Combined_Key'].str.lower().str.contains(str(row['province']).lower(), na=False).any():
         return location_df[location_df['Combined_Key'].str.lower().str.contains(str(row['province']).lower())].iloc[0]['Combined_Key']
    elif location_df['Combined_Key'].str.lower().str.contains(row['country'].lower()).any():
        return 'Unknown, ' + row['country']
# simply use this function on merged train_dataset as input_df,
# as well as dataframe from location.csv
def process_outlier(input_df, location_df):
    #1 remove date confirmed after June 2020
    input_df['date_confirmation'] = pd.to_datetime(
        input_df['date_confirmation'], format='%d.%m.%Y', errors='coerce')
    df = input_df[input_df['date_confirmation'] < '2020-06-01']
    #2 remove rows with abnormal Case-Fatality ratio
    lower, upper = get_case_fatality_range(location_df)
    df = df[(df['Case-Fatality_Ratio'] > lower) &
            (df['Case-Fatality_Ratio'] < upper)]

    return df

# function to calculate filter range
def get_case_fatality_range(location_df):
    mean_val = location_df['Case-Fatality_Ratio'].mean()
    sd = location_df['Case-Fatality_Ratio'].std()
    lower_bound = mean_val-3*sd
    upper_bound = mean_val+3*sd
    if (lower_bound < 0):
        lower_bound = 0
    return (lower_bound, upper_bound)

In [295]:

cases_df = pd.read_csv('../data/cases_train.csv')

cases_df.drop('additional_information',axis = 1,inplace = True)
cases_df.drop('source',axis = 1,inplace = True)

test_df = pd.read_csv('../data/cases_test.csv')

test_df.drop('additional_information',axis = 1,inplace = True)
test_df.drop('source',axis = 1,inplace = True)

location_df = pd.read_csv('../data/location.csv')

clean_cases_df = clean_data(cases_df)

clean_test_df = clean_data(test_df)

location_df = transform_counties(location_df)


C:\Python39\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
<ipython-input-78-54f66620475f>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_age_empty_cases['age'] = non_age_empty_cases.age.astype(str)
<ipython-input-78-54f66620475f>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

In [296]:
location_df['Incidence_Rate'] = location_df['Incidence_Rate'].fillna(location_df.groupby('Country_Region')['Incidence_Rate'].transform('mean'))
location_df['Case-Fatality_Ratio'] = location_df['Case-Fatality_Ratio'].fillna(location_df.groupby('Country_Region')['Case-Fatality_Ratio'].transform('mean'))
location_df['Active'] = location_df['Active'].fillna(location_df.groupby('Country_Region')['Active'].transform('mean'))

In [297]:
agg = location_df.groupby('Country_Region').agg(np.mean)

In [298]:
col = agg.loc['Colombia']
ukr = agg.loc['Ukraine']
russ = agg.loc['Russia']
mex = agg.loc['Mexico']
swe = agg.loc['Sweden']

In [232]:
col

Lat                        5.349139
Long_                    -74.190797
Confirmed              22981.757576
Deaths                   728.454545
Recovered              19020.757576
Active                  3232.545455
Incidence_Rate          1218.239013
Case-Fatality_Ratio        3.101681
Name: Colombia, dtype: float64

In [299]:

location_df.loc[-1] = ['Unknown', col.Lat, col.Long_, col.Confirmed, col.Deaths, col.Recovered, col.Active, col.Incidence_Rate, col['Case-Fatality_Ratio'], 'Colombia', 'Unknown, Colombia', np.nan]  # adding a row
col = ukr
location_df.loc[-2] = ['Unknown', col.Lat, col.Long_, col.Confirmed, col.Deaths, col.Recovered, col.Active, col.Incidence_Rate, col['Case-Fatality_Ratio'], 'Ukraine', 'Unknown, Ukraine', np.nan]  # adding a row
col = russ
location_df.loc[-3] = ['Unknown', col.Lat, col.Long_, col.Confirmed, col.Deaths, col.Recovered, col.Active, col.Incidence_Rate, col['Case-Fatality_Ratio'], 'Russia', 'Unknown, Russia', np.nan]  # adding a row
col = mex
location_df.loc[-4] = ['Unknown', col.Lat, col.Long_, col.Confirmed, col.Deaths, col.Recovered, col.Active, col.Incidence_Rate, col['Case-Fatality_Ratio'], 'Mexico', 'Unknown, Mexico', np.nan]  # adding a row
col = swe
location_df.loc[-5] = ['Unknown', col.Lat, col.Long_, col.Confirmed, col.Deaths, col.Recovered, col.Active, col.Incidence_Rate, col['Case-Fatality_Ratio'], 'Sweden', 'Unknown, Sweden', np.nan]  # adding a row

In [307]:
location_df.index = location_df.index+5
location_df.sort_index()

,Province_State,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incidence_Rate,Case-Fatality_Ratio,Country_Region,Combined_Key,Last_Update
5,Unknown,59.680643,15.736519,4201.761905,279.285714,0.000000,3922.476190,786.498318,5.768545,Sweden,"Unknown, Sweden",NaN
6,Unknown,21.720234,-100.527288,21691.281250,2289.312500,18317.312500,1084.656250,563.132233,11.173238,Mexico,"Unknown, Mexico",NaN
7,Unknown,54.546312,62.120860,13167.650602,232.168675,10886.084337,2049.397590,764.722354,1.397390,Russia,"Unknown, Russia",NaN
8,Unknown,48.845430,30.781274,6557.333333,132.777778,2950.777778,3473.777778,427.840823,2.067723,Ukraine,"Unknown, Ukraine",NaN
9,Unknown,5.349139,-74.190797,22981.757576,728.454545,19020.757576,3232.545455,1218.239013,3.101681,Colombia,"Unknown, Colombia",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3958,NaN,31.952200,35.233200,35003.000000,253.000000,23446.000000,11304.000000,686.142828,0.722795,West Bank and Gaza,West Bank and Gaza,2020-09-20 04:22:56
3959,NaN,24.215500,-12.885800,10.000000,1.000000,8.000000,1.000000,1.674116,10.000000,Western Sahara,Western Sahara,2020-09-20 04:22:56
3960,NaN,15.552727,48.516388,2026.000000,585.000000,1221.000000,220.000000,6.792738,28.874630,Yemen,Yemen,2020-09-20 04:22:56
3961,NaN,-13.133897,27.849332,14070.000000,330.000000,13365.000000,375.000000,76.534126,2.345416,Zambia,Zambia,2020-09-20 04:22:56


In [301]:
def impute_confirmation(X):
    X_1 = X.copy()
    X_1['date_confirmation'] = X_1['date_confirmation'].fillna('31.05.2020')
    return X_1

In [302]:
def encode_confirmation(X):
    X['date_confirmation'] = pd.to_datetime(X['date_confirmation'],format='%d.%m.%Y', errors='coerce')
    X['date_confirmation'] = X.apply(lambda x: (datetime.now() - x['date_confirmation']).days, axis=1)
    return X

In [303]:
def impute_age(X):
    null_data = X[X.isnull().any(axis=1)]
    X['age'] = X['age'].fillna(X['age'].mean())
    return X

In [304]:
def clean_combined(X):
    X['combined_key'].replace(', ', '_', regex=True, inplace=True)
    return X

In [318]:
merged_dataset_train = merge_df_location(clean_cases_df, location_df)

merged_dataset_train = process_outlier(merged_dataset_train, location_df)

merged_test = merge_df_location(clean_test_df, location_df)

In [319]:
merged_test = impute_confirmation(merged_test)
merged_dataset_train = impute_confirmation(merged_dataset_train)


In [320]:
merged_test = merged_test.drop(['outcome',"Lat", "Long_", "province", "country", "Combined_Key", "Province_State", "Last_Update", "longitude", "latitude"], axis=1)

In [321]:
merged_test = encode_confirmation(merged_test)
merged_test = impute_age(merged_test)
merged_test = clean_combined(merged_test)

In [322]:
merged_dataset_train = merged_dataset_train.drop(["Lat", "Long_", "province", "country", "Combined_Key", "Province_State", "Last_Update", "longitude", "latitude"], axis=1)
merged_dataset_train = encode_confirmation(merged_dataset_train)
merged_dataset_train = impute_age(merged_dataset_train)
merged_dataset_train = clean_combined(merged_dataset_train)

In [325]:
merged_df = merged_test[merged_test.isna().any(axis=1)]
merged_df.to_csv("../results/exceptions.csv")

In [324]:
  merged_dataset_train.to_csv("../results/cases_train_processed.csv")
  merged_test.to_csv("../results/cases_test_processed.csv")